In [1]:
library(tidyverse)
library(magrittr)
library(betareg)
library(broom)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.0     ✔ purrr   0.2.5
✔ tibble  2.0.1     ✔ dplyr   0.7.8
✔ tidyr   0.8.2     ✔ stringr 1.3.1
✔ readr   1.3.1     ✔ forcats 0.3.0
Warning message:
“package ‘tibble’ was built under R version 3.5.2”── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘magrittr’

The following object is masked from ‘package:purrr’:

    set_names

The following object is masked from ‘package:tidyr’:

    extract



In [2]:
load('../data/artsengagement.Rda')
load('../data/open_response_data/labeled_topic_prevs.Rda')
source('../scripts/select_helpers.R')
ls()

[1] "all_arts"   "all_levels" "all_yrs"    "df"         "thetas"

## Arts Identity Merging

In [3]:
df %>% select(contains('sr_identity')) %>% names

[1] "sr_identity_musician"         "sr_identity_visart"          
 [3] "sr_identity_dancer"           "sr_identity_cwriter"         
 [5] "sr_identity_film"             "sr_identity_gdesigner"       
 [7] "sr_identity_architect"        "sr_identity_actor"           
 [9] "sr_identity_designer"         "sr_identity_artsorg1"        
[11] "sr_identity_artsorg2"         "sr_identity_communityarts"   
[13] "sr_identity_studyarts"        "sr_identity_artsoutsidemajor"
[15] "sr_identity_buy"              "sr_identity_attendyearly"    
[17] "sr_identity_attendmonthly"    "sr_identity_attendweekly"    
[19] "sr_identity_makemusic"        "sr_identity_makevisart"      
[21] "sr_identity_dance"            "sr_identity_cwrite"          
[23] "sr_identity_makefilms"        "sr_identity_design"          
[25] "sr_identity_construct"        "sr_identity_perform"

In [4]:
identity <- df %>% select('sr_identity_musician','sr_identity_visart','sr_identity_dancer','sr_identity_cwriter',
              'sr_identity_film','sr_identity_gdesigner','sr_identity_architect','sr_identity_actor',
              'sr_identity_designer','sr_identity_studyarts', 'sr_identity_makemusic','sr_identity_makevisart',
              'sr_identity_dance','sr_identity_cwrite','sr_identity_makefilms','sr_identity_perform')

In [5]:
identity_agg <- c()
for(i in seq(nrow(identity))) {
    artist <- NA
    for(j in seq(identity)) {
        if (!is.na(identity[i,j]) & identity[i,j] == "Yes") {
            artist <- 1
            break
        }
        if (!is.na(identity[i,j]) & identity[i,j] == "No") {
            artist <- 2
        }
    }
    identity_agg %<>% c(artist)
}

df$sr_identity_artist <- factor(identity_agg, labels=c("Yes","No"))

In [6]:
motiv_demos <- df %>% select(contains('sr_motivation')) %>% names
identity_demos <- df %>% select(contains('sr_identity')) %>% names
participation_demos <- df %>% select(all_arts('participation')) %>% select(starts_with('sr')) %>% names
real_demos <- c('ethnic_group', 'sex', 'school', 'parented', 'income', 'hstype', 'hssize',
           'hslocation', 'hs_arts_freq', 'hs_encouragement', 'hs_required', 'hs_fees',
           'so_childhood1', 'so_childhood3', 'artsincollege', 'so_childhood5', 'sr_participated',
               'sr_highestdegreeplanned')
demo_groups <- c('real_demos','motiv_demos','identity_demos','participation_demos')

In [7]:
demo_groups <- list(real_demos, motiv_demos, identity_demos, participation_demos)

#### Replace 0 values with small nonzero value bc these categories were manually created

In [8]:
thetas[[9]] %>% select(contains('Role')) %>% head

Played a Role,Didn't Play a Role
0.07166238,0.00000000
0.07834615,0.00000000
0.04303464,0.00000000
0.00000000,0.07846073
0.12374583,0.00000000
0.07635667,0.00000000


In [9]:
zeroval <- 0.0001
for(i in seq(nrow(thetas[[9]]))) {
    if (thetas[[9]][i,2] == 0) {
        thetas[[9]][i,2] = zeroval
        thetas[[9]][i,11] = thetas[[9]][i,11] - zeroval
    }
    if (thetas[[9]][i,11] == 0) {
        thetas[[9]][i,11] = zeroval
        thetas[[9]][i,2] = thetas[[9]][i,2] - zeroval
    }
}

In [10]:
thetas[[9]] %>% select(contains('Role')) %>% head

Played a Role,Didn't Play a Role
0.07156238,0.00010000
0.07824615,0.00010000
0.04293464,0.00010000
0.00010000,0.07836073
0.12364583,0.00010000
0.07625667,0.00010000


----
----
----
----
## Grouped Demos

In [11]:
sig_contrasts.demos <- c()
for(k in c(4,8)) {
    for(d in seq(demo_groups)) {
        question_name <- names(thetas)[k]
        merged <- merge(thetas[[question_name]], select(df, key, demo_groups[[d]]), by = 'key')
        merged <- merged[-1]
        for(l in seq(demo_groups[[d]])) {
            ## NA the lonely eggs
            merged[[demo_groups[[d]][l]]][merged[[demo_groups[[d]][l]]] %in% 
                             (merged[[demo_groups[[d]][l]]] %>% table %>% tidy %>% filter(n < 25) %>% pull(1))] <- NA
            merged[[demo_groups[[d]][l]]] %<>% droplevels
            if (nlevels(merged[[demo_groups[[d]][l]]]) < 2)
                merged %<>% select(-matches(demo_groups[[d]][l]))
        }
        # now we only have good demos, and all topics
        topics <- merged[1:(ncol(thetas[[question_name]])-1)]
        demo <- merged[ncol(thetas[[question_name]]):ncol(merged)]

        for(i in seq(topics)) {
            temp <- bind_cols(topics[i], demo)

            names(temp) <- c('topic', names(demo))

            beta.out <- betareg(topic ~ ., temp)#, na.action = na.exclude)
            df.residual <- beta.out$df.residual
            r.squared <- beta.out$pseudo.r.squared

            options(warn=-1)
#             beta.safe <- beta.out
            beta.out %<>% tidy %>% filter(component == 'mean') %>% 
                                        select(-component) %>% filter(term != '(Intercept)')
            options(warn=0)

            #em.out <- emmeans(lm.out, ~ demo)
            #contrast.out <- contrast(em.out, method="del.eff") %>% summary

            beta.out$p.value %<>% p.adjust(method='holm')#, n=(ncol(topics)*nrow(beta.out)))

            beta.out %<>% filter(p.value < 0.05) %>% mutate(topic = names(topics[i]), 
                                                            question = question_name,
                                                            residual_df = df.residual,
                                                            r_sq = r.squared)

            if(nrow(beta.out) > 0) {
                options(warn=-1)
                sig_contrasts.demos %<>% bind_rows(beta.out)
                options(warn=0)
            }
        }
    }
}

In [12]:
split_list <- gregexpr("[A-Z]|[0-9]", sig_contrasts.demos$term)
demo_list <- c()
level_list <- c()
for(i in seq(sig_contrasts.demos$term)) {
    demo_list %<>% c(substr(sig_contrasts.demos$term[i], 1, split_list[[i]] - 1))
    level_list %<>% c(substr(sig_contrasts.demos$term[i], split_list[[i]],
                             nchar(sig_contrasts.demos$term[i])))
}
sig_contrasts.demos$demo <- demo_list
sig_contrasts.demos$level <- level_list
sig_contrasts.demos$term <- NULL

empty_list <- sig_contrasts.demos %>% filter(demo == '')
split_list <- gregexpr("[0-9]", empty_list$level)
demo_list <- c()
level_list <- c()
for(i in seq(empty_list$level)) {
    demo_list %<>% c(substr(empty_list$level[i], 1, split_list[[i]] - 1))
    level_list %<>% c(substr(empty_list$level[i], split_list[[i]],
                             nchar(empty_list$level[i])))
}
empty_list$demo <- demo_list
empty_list$level <- level_list
sig_contrasts.demos %<>% bind_rows(empty_list)
sig_contrasts.demos %<>% filter(demo != '')

In [13]:
sig_contrasts.demos %>% nrow
sig_contrasts.demos

[1] 82

estimate,std.error,statistic,p.value,topic,question,residual_df,r_sq,demo,level
0.9933077,0.1917906,5.179128,6.687779e-06,More Confident & Social,behavior2,30,0.4515716,ethnic_group,Asian
-0.8221978,0.1416449,-5.804642,2.064112e-07,More Confident & Social,behavior2,30,0.4515716,sex,Female
-0.9405522,0.2549704,-3.688869,6.081847e-03,More Confident & Social,behavior2,30,0.4515716,school,Ross School of Business
-1.7024398,0.4044585,-4.209183,7.432575e-04,More Confident & Social,behavior2,30,0.4515716,hssize,More than 3001
1.3042886,0.3311390,3.938795,2.292976e-03,More Confident & Social,behavior2,30,0.4515716,hslocation,Rural
-0.7694124,0.1454879,-5.288497,3.823086e-06,More Confident & Social,behavior2,30,0.4515716,hs_arts_freq,Frequently
0.5763670,0.1837866,3.136067,4.451987e-02,More Confident & Social,behavior2,30,0.4515716,hs_encouragement,Yes
0.4439039,0.1422351,3.120915,4.507247e-02,More Confident & Social,behavior2,30,0.4515716,hs_fees,Yes
-1.0383347,0.2781618,-3.732845,5.869201e-03,Deeper Understanding of Art & The World,behavior2,30,0.4455261,school,Ross School of Business
0.9149535,0.1912546,4.783955,5.500143e-05,Deeper Understanding of Art & The World,behavior2,30,0.4455261,sr_highestdegreeplanned,Master's degree


## 4 & 8 investigation

In [19]:
na_medians <- c()
na_means <- c()
for(k in seq(thetas)) {
    question_name <- names(thetas)[k]
    merged <- merge(thetas[[question_name]], select(df, key, ungrouped_demos), by = 'key')
    merged <- merged[-1] #remove key
    for(l in seq(ungrouped_demos)) {
        ## NA the lonely eggs
        merged[[ungrouped_demos[l]]][merged[[ungrouped_demos[l]]] %in% 
                         (merged[[ungrouped_demos[l]]] %>% table %>% tidy %>% filter(n < 25) %>% pull(1))] <- NA
        merged[[ungrouped_demos[l]]] %<>% droplevels
        if (nlevels(merged[[ungrouped_demos[l]]]) < 2)
            merged %<>% select(-matches(ungrouped_demos[l]))
    }
    
    # now we only have good demos, and all topics
    topics <- merged[1:(ncol(thetas[[question_name]])-1)]
    demo <- merged[ncol(thetas[[question_name]]):ncol(merged)]
    
    na_medians %<>% c(demo %>% sapply(function(x)(sum(is.na(x)))/nrow(demo)) %>% as.vector %>% median)
    na_means %<>% c(demo %>% sapply(function(x)(sum(is.na(x)))/nrow(demo)) %>% as.vector %>% mean)                                   
}

In [20]:
data.frame(na_means, na_medians, seq(12))

na_means,na_medians,seq.12.
0.46329111,0.57259528,1
0.56369913,0.72809335,2
0.57852594,0.73376335,3
0.28515532,0.32419355,4
0.41644596,0.52490315,5
0.27844881,0.31980116,6
0.03875380,0.04395604,7
0.06760413,0.04339623,8
0.06907355,0.03201970,9
0.07678826,0.04016478,10


### Nothing strange here ^^

## Ungrouped Demos for comparison

In [14]:
# ungrouped_demos <- c(demo_groups[[1]], demo_groups[[2]], demo_groups[[3]], demo_groups[[4]])
ungrouped_demos <- c(demo_groups[[1]], demo_groups[[2]], 'sr_identity_artist', demo_groups[[4]])
# ungrouped_demos <- c(demo_groups[[1]], demo_groups[[2]], demo_groups[[4]])

In [15]:
sig_contrasts.demos_no_grp <- c()
betaregs <- c()
bad_input <- c(4,8,Inf)
idx = 1
for(k in seq(thetas)) {
#     if (k != bad_input[idx]) {next}
    if (k == bad_input[idx]) {
        idx = idx + 1
        next
    }
    question_name <- names(thetas)[k]
    merged <- merge(thetas[[question_name]], select(df, key, ungrouped_demos), by = 'key')
    merged <- merged[-1] #remove key
    for(l in seq(ungrouped_demos)) {
        ## NA the lonely eggs
        merged[[ungrouped_demos[l]]][merged[[ungrouped_demos[l]]] %in% 
                         (merged[[ungrouped_demos[l]]] %>% table %>% tidy %>% filter(n < 25) %>% pull(1))] <- NA
        merged[[ungrouped_demos[l]]] %<>% droplevels
        if (nlevels(merged[[ungrouped_demos[l]]]) < 2)
            merged %<>% select(-matches(ungrouped_demos[l]))
    }
    
    # now we only have good demos, and all topics
    topics <- merged[1:(ncol(thetas[[question_name]])-1)]
    demo <- merged[ncol(thetas[[question_name]]):ncol(merged)]
    
    for(i in seq(topics)) {        
        temp <- bind_cols(topics[i], demo)

        names(temp) <- c('topic', names(demo))

        beta.out <- betareg(topic ~ ., temp)#, na.action = na.exclude)
        df.residual <- beta.out$df.residual
        r.squared <- beta.out$pseudo.r.squared
        
        options(warn=-1)
            beta.out %<>% tidy %>% filter(component == 'mean') %>% 
                                    select(-component) %>% filter(term != '(Intercept)')
        options(warn=0)

        #em.out <- emmeans(lm.out, ~ demo)
        #contrast.out <- contrast(em.out, method="del.eff") %>% summary

        beta.out$p.value %<>% p.adjust(method='holm')#, n=(ncol(topics)*nrow(beta.out)))

        beta.out %<>% filter(p.value < 0.05) %>% mutate(topic = names(topics[i]), 
                                                        question = question_name,
                                                        residual_df = df.residual,
                                                        r_sq = r.squared)

        if(nrow(beta.out) > 0) {
            options(warn=-1)
                sig_contrasts.demos_no_grp %<>% bind_rows(beta.out)
            options(warn=0)
        }
    }
}

In [16]:
backup <- sig_contrasts.demos_no_grp

In [17]:
split_list <- gregexpr("[A-Z]|[0-9]", sig_contrasts.demos_no_grp$term)
demo_list <- c()
level_list <- c()
for(i in seq(sig_contrasts.demos_no_grp$term)) {
    demo_list %<>% c(substr(sig_contrasts.demos_no_grp$term[i], 1, split_list[[i]] - 1))
    level_list %<>% c(substr(sig_contrasts.demos_no_grp$term[i], split_list[[i]],
                             nchar(sig_contrasts.demos_no_grp$term[i])))
}
sig_contrasts.demos_no_grp$demo <- demo_list
sig_contrasts.demos_no_grp$level <- level_list
sig_contrasts.demos_no_grp$term <- NULL

empty_list <- sig_contrasts.demos_no_grp %>% filter(demo == '')
split_list <- gregexpr("[0-9]", empty_list$level)
demo_list <- c()
level_list <- c()
for(i in seq(empty_list$level)) {
    demo_list %<>% c(substr(empty_list$level[i], 1, split_list[[i]] - 1))
    level_list %<>% c(substr(empty_list$level[i], split_list[[i]],
                             nchar(empty_list$level[i])))
}
empty_list$demo <- demo_list
empty_list$level <- level_list
sig_contrasts.demos_no_grp %<>% bind_rows(empty_list)
sig_contrasts.demos_no_grp %<>% filter(demo != '')

In [18]:
sig_contrasts.demos_no_grp %>% nrow
sig_contrasts.demos_no_grp$question %>% table

[1] 294

.
        barriers           change       childhood2       definition 
              11                9               30                6 
            feel sr_aftercollege4   sr_development   sr_othergrowth 
              30               24               31               19 
         sr_role       sr_society 
              69               65 

## Saving

In [25]:
qns <- sig_contrasts.demos_no_grp %>% pull(question) %>% unique

In [26]:
qns

[1] "barriers"         "change"           "definition"       "childhood2"      
 [5] "feel"             "sr_aftercollege4" "sr_development"   "sr_othergrowth"  
 [9] "sr_role"          "sr_society"

In [29]:
for(i in seq(qns)) {
    write.csv(x = sig_contrasts.demos_no_grp %>% filter(question == qns[i]) %>% arrange(demo),
              file = paste0('newdemos/', qns[i], '_demographics.csv'))
}